# 1. Dynamic Food Scene Explorer Agent

## Dependencies and necessary API keys

In [ ]:
!pip install -U langchain langchain_google_genai langchain-community langchainhub tavily-python langchain_core langsmith

In [ ]:
import os
from google.colab import userdata

# Load the secret keys from Colab's secret manager into the environment
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

# Testing LLM (The Brain)
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
print("LLM (Brain) Initialized Successfully.")

# testing search Tool (The Eyes)
from langchain_community.tools.tavily_search import TavilySearchResults
search_tool = TavilySearchResults()
tools = [search_tool]

# a quick test search
results = search_tool.invoke({"query": "Best biryani in Lucknow according to food blogs"})
print("Search Tool (Eyes) Test Successful.")
print("A sample search result:", results[0]['content'])

LLM (Brain) Initialized Successfully.
Search Tool (Eyes) Test Successful.
A sample search result: March 1, 2022 - Awadhi cuisine is known for its delicate flavours, mild aromas and nafasat (refinement). And all these nuances apply to its famous biryani, or rather pulao—yes, after umpteen debates on the difference between the two, we now know that Lakhnawi biryani is actually pulao.


## Assembling the Agent

Creating core logic that allows LLM to use the tools we provide. Building ReAct Agent (Reasoning and Acting)
1. Defining toolkit: search tool to get blogs relevant to query
2. Create Agent's Brain: LLM with sophisticated prompt that teaches it how to reason about which tool to use
3. Executer: wrap everything in an AgentExecuter (an engine that runs the agent's "think-act-observe" loop)

In [ ]:
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import PromptTemplate

## define toolkit again (for comilation at once place), for now its just search tool
tools = [search_tool]

## custom prompt for our use case
prompt_template_str = """
Answer the following questions as best you can. You are a world-class food guide.
Your goal is to find the best local dishes in a given city and the specific places to eat them. Also provide 1 thing most people say about this dish. Also find out google rating for the restaurent and how many people rate.

You have access to the following tools:
{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do.
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)

Thought: I have gathered all the necessary information, including dishes, specific restaurant names, locations, and their ratings. I am now ready to give the final answer.
Final Answer: [Your final answer must be a well-structured list. For each dish, you MUST provide the recommended place, its location (most popular, with most reviews and number of people rate), and its google rating (with number of people rate, if multiple location, rating based on highest people rating) if available. Format it nicely.]

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""
prompt = PromptTemplate.from_template(prompt_template_str)

## create agent: bind LLM with prompt
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

##create agent executer: runtime for agent, make agent think, use tool and respond
agent_executer = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

## Testing out model

In [ ]:
## testing
response = agent_executer.invoke({"input": "tell me top 3 pizza places of Bengaluru"})

print("\n--- Final Answer ---\n")
print(response["output"])



> Entering new AgentExecutor chain...
Action: tavily_search_results_json
Action Input: best pizza places Bengaluru with Google ratings and reviews and popular dishes[{'title': '9 Best Pizza Places In Bengaluru For Cheesiest Pizzas', 'url': 'https://curlytales.com/web-stories/9-best-pizza-places-in-bengaluru-for-cheesiest-pizzas/', 'content': "Are you looking for some of the best places in Bangalore that are known for serving the most delicious pizzas loaded with lots of cheese and toppings? Here's a list of all the places .", 'score': 0.98554}, {'title': 'Pick The Best Pizza in Bangalore From These Places & Binge-ho!', 'url': 'https://www.whatsuplife.in/bengaluru/blog/best-top-pizza-places-restaurants-bangalore-bengaluru', 'content': 'Top 7 Pizza Places in Bangalore That You MUST Try.Come on over to The Pizza Bakery: celebrating Neapolitan-style, wood-fired sourdough pizzas , stuffed garlic bread and a whole lot more! The place was the Winner of ‘Best Pizza ’ at the Times Food Awards